In [ ]:
pip install pandas


In [ ]:
pip install pydotplus

In [ ]:
pip install matplotlib

In [ ]:
pip install numpy

In [ ]:
pip install graphviz

In [ ]:
pip install sklearn

In [ ]:
pip install xlrd

In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
import sklearn
from IPython.display import Image  
import pydotplus
import matplotlib
pd.set_option('display.max_rows', 500)


Andiamo a importare i dati, utilizzando questo metodo per specificare il percorso. Nella funzione read_excel possiamo andare a specificare il missing value, che verrà tradotto con un NaN.

I dati devono quindi essere convertiti in un dataframe.

In [ ]:

data = pd.read_excel(r'C:\Users\giuli\Documents\GitHub\Business_Intelligence_ES4_DecisionTree\Users.xls', na_values='?')
df = pd.DataFrame(data)
df[df['Workclass'].isna()]

Andiamo a sostituire i dati mancanti con i dati della colonna più frequenti (ossia la moda). l'output di df.mode ha due righe perché nella colonna 'Age' sono presenti due mode. 

In [ ]:
df.mode()

andiamo a fillare tutti gli NaN con la moda della rispettiva colonna

In [ ]:
df = df.fillna(df.mode().iloc[0])
df[df['Workclass'].isna()]
df.dtypes
df_enc = df

for col in df_enc.columns:
    df_enc[col]=df_enc[col].astype('category')
    df_enc[col]=df_enc[col].cat.codes

df_enc['Age']=df_enc['Age'].astype('int64')

df_enc

In [ ]:
dfX= df_enc.iloc[:, 0:9]
dfY = df_enc.iloc[:, 9]

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=5)
clf = clf.fit(dfX, dfY)
tree.plot_tree(clf, feature_names=df_enc.columns)
matplotlib.pyplot.show()


In [ ]:
data_topredict = pd.read_excel(r'C:\Users\giuli\Documents\GitHub\Business_Intelligence_ES4_DecisionTree\Prospects.xls', na_values='?')
df_toPredict= pd.DataFrame(data_topredict)

In [ ]:
df_toPredict.head()

In [ ]:
for col in df_toPredict.columns:
    df_toPredict[col] = df_toPredict[col].astype('category')
    df_toPredict[col] = df_toPredict[col].cat.codes

df_toPredict['Age'] = df_toPredict['Age'].astype('int64')

In [ ]:
predictions = clf.predict(df_toPredict)

In [ ]:
predictions[33]

In [ ]:
clf.score(dfX, dfY)

Provo a fare un k-fold cross validation. Uso la classe StratifieKFold per creare l'oggetto folds. Questo mi permetterà di stratificare il dataset.

In [ ]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits = 20)

ho creato l'oggetto folds che mi permette di stratificare il dataset. Una volta splittato con il metodo split avro degli indici (degli iteratori) che vanno usati all'interno del ciclo for per selezionare il dataset di train e quello di test, esattamente come nel codice sotto

In [ ]:
splits =folds.split(dfX, dfY)
clf_k = tree.DecisionTreeClassifier(max_depth=20)
score = []
for train_index, test_index in splits:
    
    kX = dfX.iloc[train_index, :]
    kY = dfY[train_index]
    
    clf_k = clf_k.fit(kX, kY)
    
    k_test_X = dfX.iloc[test_index, :]
    k_test_Y = dfY.iloc[test_index]
    score.append(clf_k.score(k_test_X, k_test_Y))
    
print(score)


Tuttavia posso fare la stessa identica cosa con il metodo cross_val_score. Basta che gli passo il costruttore del modello che voglio testare, i dati e i responsi. Il numero di k folds fissato è 5 ma può variare

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(tree.DecisionTreeClassifier(max_depth = 7), dfX, dfY).mean()

andiamo a vedere come varia la precisione del decision tree classifier al variare della sua profondità massima:

In [ ]:
scores = []
for i in range(1,50,1):
    scores.append(cross_val_score(tree.DecisionTreeClassifier(max_depth=i), dfX, dfY).mean())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(1,50,1), scores)
ax.grid()